In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
import commonmodules as cm

filename = "N2H2_VVdata_3variables.xlsx"
df = pd.read_excel(filename)
debug = False

In [ ]:
from sklearn.preprocessing import MinMaxScaler

x = df[['v', 'w', 'T(K)']].values
y = df[['k(cm^3/s)']].values

scalerx = MinMaxScaler()
scalerx.fit(x)
x_s = scalerx.transform(x)

scalery = MinMaxScaler()
scalery.fit(y)
y_s = scalery.transform(y)

if debug:
    for i, ys in enumerate(y_s):
        print(ys, y[i])
    for i, xs in enumerate(x_s):
        print(xs, x[i])

In [ ]:
from tensorflow import keras
import tensorflow as tf

import tensorflow.keras.optimizers as tko
import tensorflow.keras.activations as tka
import tensorflow.keras.losses as tkl
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model

from sklearn.model_selection import train_test_split

def buildmodel(modelshape):
    model = keras.Sequential()
    model.add(keras.layers.Dense(units = 3, activation = 'linear', input_shape=[3]))

    for n in modelshape:
        model.add(keras.layers.Dense(units = n, activation = 'relu'))

    model.add(keras.layers.Dense(units = 1, activation = 'linear'))
    model.compile(loss='mse', optimizer="adam", metrics='mse')

    return model


train_x, test_x, train_y, test_y = train_test_split(x_s, y_s, test_size=0.33, random_state=42)

modelshape = [32, 32, 32, 32]
epochs = 10
batch_size = 50

model = buildmodel(modelshape)
history = model.fit(train_x, train_y, epochs=epochs,  batch_size=batch_size, \
    verbose=1)



In [ ]:
from sklearn import metrics

pred_y = model.predict(test_x)
plt.scatter(test_y, pred_y)
plt.show()
print("Test MSE: %10.5f"%(metrics.mean_absolute_error(test_y, pred_y)))
print(" Test R2: %10.5f"%(metrics.r2_score(test_y, pred_y)))

pred_y = model.predict(train_x)
plt.scatter(train_y, pred_y)
plt.show()
print("Training MSE: %10.5f"%(metrics.mean_absolute_error(train_y, pred_y)))
print(" Training R2: %10.5f"%(metrics.r2_score(train_y, pred_y)))
